In [1]:
import numpy as np
import pandas as pd


In [2]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer

lemtzer = WordNetLemmatizer()

def lemmatize_stemming(text):
    return lemtzer.lemmatize(text, pos='v')

# Write a function to perform the pre processing steps on the entire dataset
def preprocess(text):
    result=[]
    for token in simple_preprocess(text) :
        if token not in STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

C:\Users\USER\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
import nltk

In [4]:
document_num = 50
doc_sample = 'This disk has failed many times. I would like to get it replaced.'

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))

Original document: 
['This', 'disk', 'has', 'failed', 'many', 'times.', 'I', 'would', 'like', 'to', 'get', 'it', 'replaced.']


Tokenized and lemmatized document: 
['disk', 'fail', 'time', 'like', 'replace']


In [5]:
bitcoin_news = pd.read_csv('bitcoin_news.csv')
bitcoin_news['Title_Summary'] = bitcoin_news['Title'] + str(' ')+ bitcoin_news['Summary']
bitcoin_news.head()

,Date,Summary,Tags,Title,Title_Summary
0,2018-09-04T08:00:04-04:00,The Tuesday edition of our daily roundup is ja...,"ATM,Blockchain,Brave,Deutsche Börse,Ethereum,G...","The Daily: Wirex Adds Ethereum, Deutsche Börse...","The Daily: Wirex Adds Ethereum, Deutsche Börse..."
1,2018-08-02T17:15:26-04:00,Japan’s SBI Group is reportedly planning to cr...,"ATM,BCH,Bitcoin,bitcoin cash,BTC,BTM,coin asse...","SBI Plans Derivatives Platform, Huobi Eyes 30%...","SBI Plans Derivatives Platform, Huobi Eyes 30%..."
2,2018-09-05T01:15:31-04:00,Non-custodial crypto trading platform Shapeshi...,"AML,anti-money laundering,Bitcoin,BTC,crypto,C...",Shapeshift Moves to Membership Model Requiring...,Shapeshift Moves to Membership Model Requiring...
3,2018-08-04T04:55:54-04:00,"Intercontinental Exchange (ICE), owner of argu...","401k,Bitcoin,Boston Consulting Group,Canada,CF...",NYSE Owner: Bitcoin Should Be in Retirement Fu...,NYSE Owner: Bitcoin Should Be in Retirement Fu...
4,2018-08-06T03:45:21-04:00,Thailand’s central bank has announced the rule...,"bank of thailand,banks,Bitcoin,Bot,BTC,commerc...",Bank of Thailand Green-Lights Financial Compan...,Bank of Thailand Green-Lights Financial Compan...


In [6]:
processed_docs  = []

for doc in bitcoin_news['Title_Summary']:
    processed_docs.append(preprocess(doc))

In [7]:
import gensim
dictionary = gensim.corpora.Dictionary(processed_docs)

In [8]:
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=100000)

In [9]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 add
1 atms
2 board
3 brave
4 brief
5 bring
6 browser
7 daily
8 edition
9 embrace
10 ethereum


In [10]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

# LDA with word2vec

In [11]:
%%time
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=6, id2word=dictionary, passes=2)
lda_model.save('lda.model')

Wall time: 12.3 s


In [12]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} Word: {}\n'.format(idx, topic))

Topic: 0 Word: 0.023*"bank" + 0.016*"mine" + 0.007*"world" + 0.006*"week" + 0.006*"central" + 0.006*"price" + 0.006*"report" + 0.006*"currency" + 0.006*"digital" + 0.006*"launch"

Topic: 1 Word: 0.012*"price" + 0.010*"digital" + 0.008*"cash" + 0.007*"value" + 0.007*"week" + 0.007*"past" + 0.006*"launch" + 0.006*"update" + 0.006*"platform" + 0.006*"year"

Topic: 2 Word: 0.015*"bank" + 0.013*"brief" + 0.012*"mine" + 0.010*"coin" + 0.009*"financial" + 0.008*"offer" + 0.008*"today" + 0.006*"daily" + 0.005*"coinbase" + 0.005*"launch"

Topic: 3 Word: 0.013*"announce" + 0.012*"launch" + 0.010*"fund" + 0.009*"securities" + 0.008*"bank" + 0.007*"mine" + 0.007*"digital" + 0.007*"cash" + 0.006*"offer" + 0.006*"coin"

Topic: 4 Word: 0.024*"cash" + 0.009*"service" + 0.008*"launch" + 0.008*"japanese" + 0.007*"state" + 0.007*"network" + 0.007*"platform" + 0.007*"coin" + 0.006*"year" + 0.006*"south"

Topic: 5 Word: 0.012*"report" + 0.009*"government" + 0.008*"south" + 0.008*"financial" + 0.007*"korean

In [13]:
num = 100
unseen_document = bitcoin_news['Title_Summary'][num]
print(unseen_document)

Bitmain Launches Key Crypto Mining Facility in Texas Bitmain Technologies Ltd. is launching a crypto mining facility and blockchain data center in the U.S. state of Texas at a former aluminum smelting facility with a recently-retired coal power station. The company is reportedly getting a tax abatement phased in over 10 years.


In [14]:
# Data preprocessing step for the unseen document
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.9599984288215637	 Topic: 0.023*"bank" + 0.016*"mine" + 0.007*"world" + 0.006*"week" + 0.006*"central"


In [15]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

C:\Users\USER\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.025885 -0.040291       1        1  21.286697
0      0.010226  0.027003       2        1  19.137318
4      0.005950 -0.059917       3        1  16.240316
3      0.033534  0.016757       4        1  15.138131
2      0.016388  0.053092       5        1  14.549422
5     -0.091983  0.003356       6        1  13.648109, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
88    Default  1213.000000          bank  1213.000000  30.0000  30.0000
449   Default  1091.000000          cash  1091.000000  29.0000  29.0000
4     Default   427.000000         brief   427.000000  28.0000  28.0000
179   Default   981.000000          mine   981.000000  27.0000  27.0000
40    Default   423.000000         south   423.000000  26.0000  26.0000
18    Default   323.000000    securities   323.000000  25.0000  25.0000
209   Default   144.000000    revolution   144.000000  24.0000  24.0000
34    Default   295.000000        korean   295.000000  23.0000  23.0000
86    Default   700.000000      announce   700.000000  22.0000  22.0000
68    Default   582.000000          fund   582.000000  21.0000  21.0000
515   Default   315.000000         court   315.000000  20.0000  20.0000
132   Default   781.000000        report   781.000000  19.0000  19.0000
224   Default   464.000000         today   464.000000  18.0000  18.0000
259   Default   513.000000    government   513.000000  17.0000  17.0000
1698  Default   144.000000        ripple   144.000000  16.0000  16.0000
621   Default   151.000000       privacy   151.000000  15.0000  15.0000
8     Default   160.000000       edition   160.000000  14.0000  14.0000
1342  Default   147.000000         smart   147.000000  13.0000  13.0000
170   Default   283.000000       chinese   283.000000  12.0000  12.0000
89    Default   344.000000       central   344.000000  11.0000  11.0000
227   Default   480.000000        wallet   480.000000  10.0000  10.0000
301   Default   323.000000         value   323.000000   9.0000   9.0000
137   Default   722.000000          coin   722.000000   8.0000   8.0000
300   Default   270.000000        update   270.000000   7.0000   7.0000
94    Default   689.000000     financial   689.000000   6.0000   6.0000
314   Default   745.000000         price   745.000000   5.0000   5.0000
276   Default   372.000000      japanese   372.000000   4.0000   4.0000
541   Default   212.000000         cover   212.000000   3.0000   3.0000
109   Default   295.000000      coinbase   295.000000   2.0000   2.0000
202   Default   123.000000       twitter   123.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
407    Topic6    38.112106      nakamoto    80.863419   1.2393  -6.2201
170    Topic6   112.025269       chinese   283.879456   1.0617  -5.1419
40     Topic6   157.116226         south   423.082764   1.0010  -4.8036
259    Topic6   178.778473    government   513.953247   0.9356  -4.6745
1124   Topic6    76.468567      contract   186.873886   1.0980  -5.5237
515    Topic6   116.407433         court   315.968933   0.9930  -5.1035
268    Topic6   101.379250   regulations   268.556244   1.0174  -5.2417
272    Topic6   116.865944  transactions   333.330139   0.9435  -5.0996
132    Topic6   227.633560        report   781.374817   0.7583  -4.4329
104    Topic6   108.234581    regulatory   308.268494   0.9449  -5.1763
105    Topic6    76.274300          rule   199.576752   1.0297  -5.5263
319    Topic6   126.162209        accord   421.327423   0.7857  -5.0230
12     Topic6   119.960739          firm   468.759460   0.6286  -5.0734
94     Topic6   155.080505     financial   689.841309   0.4991  -4.8167
227    Topic6   119.022575        wallet   480.097504   0.5969  -5.0813
911    Topic6    88.720169           say   315.362549   0.7233  -5.37

# LDA with doc2vec

In [16]:
from gensim import models
import warnings
warnings.filterwarnings('ignore')

In [17]:
tfidf = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]

In [18]:
%%time
lda_model_tfidf = gensim.models.LdaMulticore(tfidf_corpus, num_topics=6, id2word=dictionary, passes=2)
lda_model_tfidf.save('lda_tfidf.model')

Wall time: 20.1 s


In [19]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.006*"mine" + 0.005*"coinbase" + 0.005*"bank" + 0.004*"offer" + 0.004*"service" + 0.004*"launch" + 0.004*"city" + 0.004*"world" + 0.004*"brief" + 0.004*"platform"
Topic: 1 Word: 0.008*"bank" + 0.006*"revolution" + 0.004*"price" + 0.004*"financial" + 0.004*"satoshi" + 0.003*"central" + 0.003*"world" + 0.003*"country" + 0.003*"service" + 0.003*"rise"
Topic: 2 Word: 0.005*"launch" + 0.005*"cash" + 0.005*"currency" + 0.004*"digital" + 0.004*"accept" + 0.004*"bank" + 0.004*"wallet" + 0.004*"mine" + 0.004*"platform" + 0.004*"service"
Topic: 3 Word: 0.006*"launch" + 0.005*"price" + 0.005*"wallet" + 0.005*"mine" + 0.004*"week" + 0.004*"fund" + 0.004*"report" + 0.004*"cash" + 0.003*"time" + 0.003*"million"
Topic: 4 Word: 0.007*"cash" + 0.004*"chinese" + 0.004*"mine" + 0.004*"bank" + 0.004*"south" + 0.004*"coin" + 0.003*"money" + 0.003*"week" + 0.003*"digital" + 0.003*"offer"
Topic: 5 Word: 0.007*"bank" + 0.006*"mine" + 0.004*"cash" + 0.004*"announce" + 0.004*"price" + 0.004*"lau

In [20]:
pyLDAvis.gensim.prepare(lda_model_tfidf, tfidf_corpus, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.006834 -0.002942       1        1  20.942888
3      0.021250 -0.024395       2        1  18.262047
1      0.002196  0.049410       3        1  17.730078
2     -0.001503 -0.002889       4        1  15.661645
5      0.021574 -0.007512       5        1  14.743407
0     -0.050352 -0.011673       6        1  12.659929, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
209   Default   40.000000    revolution   40.000000  30.0000  30.0000
360   Default   20.000000          city   20.000000  29.0000  29.0000
109   Default   52.000000      coinbase   52.000000  28.0000  28.0000
179   Default  109.000000          mine  109.000000  27.0000  27.0000
208   Default   13.000000       mcelroy   13.000000  26.0000  26.0000
212   Default   13.000000         wendy   13.000000  25.0000  25.0000
88    Default  132.000000          bank  132.000000  24.0000  24.0000
1427  Default   17.000000        circle   17.000000  23.0000  23.0000
206   Default   17.000000  expectations   17.000000  22.0000  22.0000
696   Default   48.000000        accept   48.000000  21.0000  21.0000
1054  Default   40.000000         block   40.000000  20.0000  20.0000
887   Default   30.000000       futures   30.000000  19.0000  19.0000
596   Default   36.000000          peer   36.000000  18.0000  18.0000
1017  Default   13.000000         index   13.000000  17.0000  17.0000
1     Default   11.000000          atms   11.000000  16.0000  16.0000
311   Default   13.000000         petro   13.000000  15.0000  15.0000
1819  Default    9.000000         tezos    9.000000  14.0000  14.0000
211   Default   43.000000       satoshi   43.000000  13.0000  13.0000
472   Default   38.000000      payments   38.000000  12.0000  12.0000
806   Default   18.000000      telegram   18.000000  11.0000  11.0000
1553  Default    9.000000  antonopoulos    9.000000  10.0000  10.0000
1691  Default   14.000000     lightning   14.000000   9.0000   9.0000
440   Default   23.000000       federal   23.000000   8.0000   8.0000
1303  Default   10.000000     argentina   10.000000   7.0000   7.0000
1757  Default   10.000000        mcafee   10.000000   6.0000   6.0000
1190  Default    8.000000          info    8.000000   5.0000   5.0000
40    Default   56.000000         south   56.000000   4.0000   4.0000
227   Default   76.000000        wallet   76.000000   3.0000   3.0000
1027  Default   28.000000       reserve   28.000000   2.0000   2.0000
116   Default   42.000000       largest   42.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
1383   Topic6    3.666259   corporation    9.611107   1.1030  -6.8118
596    Topic6   10.245780          peer   36.534012   0.7954  -5.7841
116    Topic6   11.422051       largest   42.398121   0.7552  -5.6754
7      Topic6   10.046567         daily   35.763569   0.7970  -5.8037
613    Topic6   10.842484       russian   41.788605   0.7176  -5.7275
1054   Topic6   10.300651         block   40.599640   0.6952  -5.7788
144    Topic6    8.253130         local   30.496141   0.7597  -6.0004
4      Topic6   11.778959         brief   52.491028   0.5724  -5.6446
179    Topic6   19.001637          mine  109.090973   0.3191  -5.1664
180    Topic6   14.499388         offer   75.333237   0.4189  -5.4369
473    Topic6    9.004325        russia   36.370464   0.6707  -5.9133
786    Topic6    7.817167        social   29.065392   0.7535  -6.0546
325    Topic6   13.751491       service   82.956917   0.2696  -5.4898
88     Topic6   16.536617          bank  132.662415  -0.0155  -5.3054
83     Topic6   11.950759         world   70.630928   0.2901  -5.6302
37     Topic6   11.668453      platform   69.894577   0.2766  -5.6541
321    Topic6   11.516425          base   72.090370   0.2326  -5.6672
273    Topic6   11.290548